In [ ]:
from netCDF4 import Dataset
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import datetime
from scipy.interpolate import griddata
from mpl_toolkits.basemap import Basemap
import os
from sklearn.linear_model import LinearRegression

In [ ]:
start_time_era5 = datetime.datetime(1900, 1, 1, 0, 0, 0)

# Границы региона, загружаем только эти данные из хранилища
lon_min_era5, lon_max_era5, lat_min_era5, lat_max_era5 = 90, 200, 67, 80

In [ ]:
# Создаем координатную сетку на основе одного из файлов. Сетка не меняется между файлами
lat1, lat2 = (90-lat_max_era5)*4, (90-lat_min_era5)*4+1
lon1, lon2 = lon_min_era5*4, lon_max_era5*4+1

file = '/mnt/hippocamp/DATA/ERA5/w10/era5_uv10m_2020-09.nc'
data = Dataset(file, 'r')

longitude = np.array(data.variables['longitude'][lon1:lon2])
latitude = np.array(data.variables['latitude'][lat1:lat2])
lon_grid, lat_grid = np.meshgrid(longitude, latitude)

In [ ]:
def get_wind(year, month, borders):
    file = f'/mnt/hippocamp/DATA/ERA5/w10/era5_uv10m_{year}-{month:02}.nc'
    data = Dataset(file, 'r')

    u10 = np.asarray(data.variables['u10'][:,lat1:lat2,lon1:lon2])
    v10 = np.asarray(data.variables['v10'][:,lat1:lat2,lon1:lon2])

    u10_upd = u10[:,(lat_max_era5-borders[0])*4:(lat_max_era5-borders[1])*4+1,
                  (borders[3]-lon_min_era5)*4:(borders[2]-lon_max_era5)*4]
    v10_upd = v10[:,(lat_max_era5-borders[0])*4:(lat_max_era5-borders[1])*4+1,
                  (borders[3]-lon_min_era5)*4:(borders[2]-lon_max_era5)*4]
    
    list_u10 = np.split(u10, u10.shape[0]/24, axis=0)
    array_u10 = np.array([np.mean(a, axis=0) for a in list_u10])

    list_v10 = np.split(v10, v10.shape[0]/24, axis=0)
    array_v10 = np.array([np.mean(a, axis=0) for a in list_v10])
    
    list_u10_upd = np.split(u10_upd, u10_upd.shape[0]/24, axis=0)
    array_u10_upd = np.array([np.mean(a, axis=0) for a in list_u10_upd])

    list_v10_upd = np.split(v10_upd, v10_upd.shape[0]/24, axis=0)
    array_v10_upd = np.array([np.mean(a, axis=0) for a in list_v10_upd])

    start_date = datetime.datetime(year, month, 1)
    finish_date = (start_date + datetime.timedelta(days=len(list_v10_upd)))
    dates = [(start_date + datetime.timedelta(days=i)).date() for i in range((finish_date - start_date).days)]
    data.close()

    return array_u10, array_v10, array_u10_upd, array_v10_upd, dates

In [ ]:
def draw_border_wind(dates, NB_130E, NB_delta_130E, dates_wind, wind_data, Ekman_data, title):
    x1 = dates
    y1 = NB_130E
    y3 = NB_delta_130E

    x3 = dates_wind
    y5 = wind_data
    y6 = Ekman_data

    fig, ax1 = plt.subplots(dpi=300)
    ax2 = ax1.twinx()

    ax3 = ax1.twinx()
    ax3.spines["right"].set_position(("outward", 60))
    ax3.spines["right"].set_visible(True)

    ax4 = ax1.twinx()
    ax4.spines["right"].set_position(("outward", 120))
    ax4.spines["right"].set_visible(True)

    line1, = ax1.plot(x1, y1, label='130E', linestyle='-', color='red')

    line3, = ax2.plot(x1, y3, label='delta130E', linestyle='--', color='red')

    line5, = ax3.plot(x3, y5, label='V_d', linestyle='-', color='black')
    line6, = ax4.plot(x1, y6, label='Ekman', linestyle='--', color='black')

    ax3.axhline(y=0, color='black', linestyle='--', linewidth=1)

    all3 = np.array(y3)
    all_wind = np.array(y5)
    all6 = np.array(y6)

    a1, a2, a3, b1, b2, b3 = (np.nanmax(all3), all_wind.max(), np.nanmax(all6),
                              np.nanmin(all3), all_wind.min(), np.nanmin(all6))

    if abs(a1/b1) > abs(a2/b2):
        new_a2 = a1 * b2 / b1
        new_b2 = b2

        new_a3 = a1 * b3 / b1
        new_b3 = b3

    elif abs(a1/b1) < abs(a2/b2):
        new_a2 = a2
        new_b2 = b1 * a2 / a1

        new_a3 = a3
        new_b3 = b1 * a3 / a1
        
    ax2.set_ylim(b1 + 0.05 * b1, a1 + 0.05 * a1)
    ax3.set_ylim(new_b2 + 0.05 * new_b2, new_a2 + 0.05 * new_a2)
    ax4.set_ylim(new_b3 + 0.05 * new_b3, new_a3 + 0.05 * new_a3)

    # lines = [line1, line2, line3, line4, line5, line6]
    # labels = [line.get_label() for line in lines]
    # ax1.legend(lines, labels, loc='upper left')

    ax1.set_ylabel('Северная граница')
    ax2.set_ylabel('Изменение северной границы')
    ax3.set_ylabel('Скорость полного потока чисто дрейфового течения')
    ax4.set_ylabel('Экмановский перенос')

    step = len(x1) // 15 + 1
    selected_indices = range(0, len(x1), step)

    selected_x = [x1[i] for i in selected_indices]
    ax1.set_xticks(selected_x)
    ax1.set_xticklabels([date.strftime('%Y-%m-%d') for date in selected_x], rotation=45)

    plt.title(title)
    plt.grid(True)
    plt.savefig(f'border_pictures_East_v2/N{title}', dpi=300, bbox_inches='tight')

In [ ]:
def make_wind(borders, months, year, start_date, finish_date, lat1=lat1, lat2=lat2, lon1=lon1, lon2=lon2):
    array_u10, array_v10 = np.empty((0, lat2-lat1, lon2-lon1)), np.empty((0, lat2-lat1, lon2-lon1))

    array_u10_upd = np.empty((0, (borders[0]-borders[1])*4+1, (borders[2]-borders[3])*4+1))
    array_v10_upd = np.empty((0, (borders[0]-borders[1])*4+1, (borders[2]-borders[3])*4+1))
    dates_era = []

    for month in months:
        array_u10_, array_v10_, array_u10_upd_, array_v10_upd_, dates_era_ = get_wind(year=year, month=month, borders=borders)
        array_u10 = np.concatenate((array_u10, array_u10_))
        array_v10 = np.concatenate((array_v10, array_v10_))
        array_u10_upd = np.concatenate((array_u10_upd, array_u10_upd_))
        array_v10_upd = np.concatenate((array_v10_upd, array_v10_upd_))
        dates_era += dates_era_

    print(array_u10.shape, array_v10.shape, array_u10_upd.shape, array_v10_upd.shape, len(dates_era))

    start_date -= datetime.timedelta(days=1)
    finish_date += datetime.timedelta(days=1)

    date_mask = [(start_date <= d <= finish_date) for d in dates_era]
    dates_wind = [d for d, keep in zip(dates_era, date_mask) if keep]
    array_u10_filtered = array_u10[date_mask,:,:]
    array_v10_filtered = array_v10[date_mask,:,:]
    array_u10_upd_filtered = array_u10_upd[date_mask,:,:]
    array_v10_upd_filtered = array_v10_upd[date_mask,:,:]

    print(array_u10_filtered.shape, array_v10_filtered.shape,
        array_u10_upd_filtered.shape, array_v10_upd_filtered.shape,
        len(dates_wind))
    
    return dates_wind, array_u10_filtered, array_v10_filtered, array_u10_upd_filtered, array_v10_upd_filtered

In [ ]:
def get_regression(U_neg, V, border):
    X1 = np.array(U_neg)
    X2 = np.array(V)
    y = np.array(border)

    mask = ~np.isnan(y)
    X = np.column_stack((X1, X2))
    X_clean = X[mask]
    y_clean = y[mask]

    model = LinearRegression()
    model.fit(X_clean, y_clean)

    y_pred = model.predict(X_clean)
    
    return y_pred, model.coef_, model.intercept_

In [ ]:
def mean_wind(array_u10_upd_filtered, array_v10_upd_filtered):
    u10_mean_neg = [-np.mean(array_u10_upd_filtered[i]) for i in range(array_u10_upd_filtered.shape[0])]
    v10_mean = [np.mean(array_v10_upd_filtered[i]) for i in range(array_v10_upd_filtered.shape[0])]
    wind_mean = [(a**2 + b**2)**0.5 for a, b in zip(u10_mean_neg, v10_mean)]
    U_neg = [a * b for a, b in zip(wind_mean, u10_mean_neg)]
    V = [a * b for a, b in zip(wind_mean, v10_mean)]

    return u10_mean_neg, v10_mean, wind_mean, U_neg, V

In [ ]:
def make_ekman(NB_delta_130E, U_neg, V):
    rho_air = 1.225
    rho_water = 1025
    f = 1.41 * 0.0001
    C = 1.3 * 0.001
    H = 10

    NB_delta_130E_km = [b * 111.3 if not np.isnan(b) else np.nan for b in NB_delta_130E]
    U_Ekman = [u * ((rho_air * C) / (rho_water * f) / H) for u in U_neg]
    U_Ekman_sm = [u * 100 for u in U_Ekman]
    V_Ekman = [v * ((rho_air * C) / (rho_water * f) / H) for v in V]
    V_Ekman_sm = [v * 100 for v in V_Ekman]

    U_Ekman_sum = [np.sum(U_Ekman[i:i+3]) for i in range(0, len(U_Ekman), 3) if len(U_Ekman[i:i+3]) == 3]
    V_Ekman_sum = [np.sum(V_Ekman[i:i+3]) for i in range(0, len(V_Ekman), 3) if len(V_Ekman[i:i+3]) == 3]
    Ekman_data_u = [60 * 60 * 24 / 1000 * u for u in U_Ekman_sum]
    Ekman_data_v = [60 * 60 * 24 / 1000 * v for v in V_Ekman_sum]

    return NB_delta_130E_km, U_Ekman, U_Ekman_sm, U_Ekman_sum, V_Ekman_sm, V_Ekman_sum, Ekman_data_u, Ekman_data_v

In [ ]:
def wind_3d(U_neg, V, NB_delta_130E_km):
    U_neg_averages = [np.mean(U_neg[i:i+3]) for i in range(0, len(U_neg), 3)]
    V_averages = [np.mean(V[i:i+3]) for i in range(0, len(V), 3)]
    df = pd.DataFrame({'a': U_neg_averages, 'b': NB_delta_130E_km})

    return U_neg_averages, V_averages, df

In [ ]:
def make_reg_coef(U_neg_averages, V_averages, NB_delta_130E_km):
    y_pred130, coef130, intercept130 = get_regression(U_neg_averages, V_averages, NB_delta_130E_km)

    NB_delta_130E_km_cleaned = list(np.array(NB_delta_130E_km)[~np.isnan(NB_delta_130E_km)])

    df130 = pd.DataFrame({'a': y_pred130, 'b': NB_delta_130E_km_cleaned})

    return df130, coef130, intercept130

Межгодовая изменчивость

In [ ]:
# 2015
year = 2015
start_date = datetime.datetime(year, 7, 23).date()
finish_date = datetime.datetime(year, 10, 9).date()
dates = [(start_date + datetime.timedelta(days=i*3)) for i in range((finish_date - start_date).days // 3 + 1)]
NB_130E = [75.25,75.75,76.25,76.375,76.75,77,76.875,77,76.875,77.125,77,77.25,77.125,77.125,
           77.125,76.875,77.125,76.875,77.25,77.375,77.25,77.125,77.25,77.375,77.125,76.25,76.125]
NB_delta_130E = [np.nan,0.5,0.5,0.125,0.375,0.25,-0.125,0.125,-0.125,0.25,-0.125,0.25,-0.125,0,0,
                 -0.25,0.25,-0.25,0.375,0.125,-0.125,-0.125,0.125,0.125,-0.25,-0.875,-0.125]

In [ ]:
border_north = 79
border_south = 73
border_east = 155
border_west = 120
borders = [border_north, border_south, border_east, border_west]

months = [7,8,9,10]

dates_wind, array_u10_filtered, array_v10_filtered, array_u10_upd_filtered, array_v10_upd_filtered = make_wind(borders, months, year, start_date, finish_date)

In [ ]:
u10_mean_neg, v10_mean, wind_mean, U_neg, V = mean_wind(array_u10_upd_filtered, array_v10_upd_filtered)
NB_delta_130E_km, U_Ekman, U_Ekman_sm, U_Ekman_sum, V_Ekman_sm, V_Ekman_sum, Ekman_data_u, Ekman_data_v = make_ekman(NB_delta_130E, U_neg, V)
U_neg_averages, V_averages, df = wind_3d(U_neg, V, NB_delta_130E_km)
print(df['a'].corr(df['b']))

In [ ]:
draw_border_wind(dates, NB_130E, NB_delta_130E_km, dates_wind, U_Ekman_sm, Ekman_data_u, str(year))

In [ ]:
# 2016
year = 2016
start_date = datetime.datetime(year, 9, 15).date()
finish_date = datetime.datetime(year, 10, 3).date()
dates = [(start_date + datetime.timedelta(days=i*3)) for i in range((finish_date - start_date).days // 3 + 1)]
NB_130E = [74.25,74.5,74,74.25,74.25,74,74.25]
NB_delta_130E = [np.nan,0.25,-0.5,0.25,0,-0.25,0.25]

In [ ]:
border_north = 76
border_south = 72
border_east = 155
border_west = 120
borders = [border_north, border_south, border_east, border_west]

months = [9,10]

dates_wind, array_u10_filtered, array_v10_filtered, array_u10_upd_filtered, array_v10_upd_filtered = make_wind(borders, months, year, start_date, finish_date)

In [ ]:
u10_mean_neg, v10_mean, wind_mean, U_neg, V = mean_wind(array_u10_upd_filtered, array_v10_upd_filtered)
NB_delta_130E_km, U_Ekman, U_Ekman_sm, U_Ekman_sum, V_Ekman_sm, V_Ekman_sum, Ekman_data_u, Ekman_data_v = make_ekman(NB_delta_130E, U_neg, V)
U_neg_averages, V_averages, df = wind_3d(U_neg, V, NB_delta_130E_km)
print(df['a'].corr(df['b']))

In [ ]:
draw_border_wind(dates, NB_130E, NB_delta_130E_km, dates_wind, U_Ekman_sm, Ekman_data_u, str(year))

In [ ]:
# 2017
year = 2017
start_date = datetime.datetime(year, 8, 13).date()
finish_date = datetime.datetime(year, 9, 24).date()
dates = [(start_date + datetime.timedelta(days=i*3)) for i in range((finish_date - start_date).days // 3 + 1)]
NB_130E = [74.25,74.625,75,74.625,75.125,75.375,75,75.5,75.875,76.5,77.125,76.875,76.625,76.375,75.625]
NB_delta_130E = [np.nan,0.375,0.375,-0.375,0.5,0.25,-0.375,0.5,0.375,0.625,0.625,-0.25,-0.25,-0.25,-0.75]

In [ ]:
border_north = 79
border_south = 73
border_east = 150
border_west = 120
borders = [border_north, border_south, border_east, border_west]

months = [8,9]

dates_wind, array_u10_filtered, array_v10_filtered, array_u10_upd_filtered, array_v10_upd_filtered = make_wind(borders, months, year, start_date, finish_date)

In [ ]:
u10_mean_neg, v10_mean, wind_mean, U_neg, V = mean_wind(array_u10_upd_filtered, array_v10_upd_filtered)
NB_delta_130E_km, U_Ekman, U_Ekman_sm, U_Ekman_sum, V_Ekman_sm, V_Ekman_sum, Ekman_data_u, Ekman_data_v = make_ekman(NB_delta_130E, U_neg, V)
U_neg_averages, V_averages, df = wind_3d(U_neg, V, NB_delta_130E_km)
print(df['a'].corr(df['b']))

In [ ]:
draw_border_wind(dates, NB_130E, NB_delta_130E_km, dates_wind, U_Ekman_sm, Ekman_data_u, str(year))

In [ ]:
# 2018
year = 2018
start_date = datetime.datetime(year, 7, 8).date()
finish_date = datetime.datetime(year, 10, 15).date()
dates = [(start_date + datetime.timedelta(days=i*3)) for i in range((finish_date - start_date).days // 3 + 1)]
NB_130E = [76.125,75.875,76.125,75.75,75.375,76,75.5,75.625,75.75,76.125,75.875,76.125,76.125,76.25,76.125,76.25,76.75,
           76.125,76.125,76.125,75.75,76,75.875,76.125,76.625,76.25,76.125,75.625,75.75,75.625,75.375,75.125,75.25,74.375]
NB_delta_130E = [np.nan,-0.25,0.25,-0.375,-0.375,0.625,-0.5,0.125,0.125,0.375,-0.25,0.25,0,0.125,-0.125,0.125,0.5,
                 -0.625,0,0,-0.375,0.25,-0.125,0.25,0.5,-0.375,-0.125,-0.5,0.125,-0.125,-0.25,-0.25,0.125,-0.875]

In [ ]:
border_north = 78
border_south = 73
border_east = 145
border_west = 125
borders = [border_north, border_south, border_east, border_west]

months = [7,8,9,10]

dates_wind, array_u10_filtered, array_v10_filtered, array_u10_upd_filtered, array_v10_upd_filtered = make_wind(borders, months, year, start_date, finish_date)

In [ ]:
u10_mean_neg, v10_mean, wind_mean, U_neg, V = mean_wind(array_u10_upd_filtered, array_v10_upd_filtered)
NB_delta_130E_km, U_Ekman, U_Ekman_sm, U_Ekman_sum, V_Ekman_sm, V_Ekman_sum, Ekman_data_u, Ekman_data_v = make_ekman(NB_delta_130E, U_neg, V)
U_neg_averages, V_averages, df = wind_3d(U_neg, V, NB_delta_130E_km)
print(df['a'].corr(df['b']))

In [ ]:
draw_border_wind(dates, NB_130E, NB_delta_130E_km, dates_wind, U_Ekman_sm, Ekman_data_u, str(year))

In [ ]:
# 2019
year = 2019
start_date = datetime.datetime(year, 7, 23).date()
finish_date = datetime.datetime(year, 10, 12).date()
dates = [(start_date + datetime.timedelta(days=i*3)) for i in range((finish_date - start_date).days // 3 + 1)]
NB_130E = [74.75,74.75,74.875,75.125,75.125,75.25,75.125,75.125,75.125,75.25,75.25,75.375,75.5,75.5,
           75.25,75.5,75.5,75,75.375,75.625,75.625,75.625,75.875,76.25,76.375,76.625,76.875,76.625]
NB_delta_130E = [np.nan,0,0.125,0.25,0,0.125,-0.125,0,0,0.125,0,0.125,0.125,0,-0.25,0.25,0,-0.5,0.375,0.25,0,0,0.25,0.375,0.125,0.25,0.25,-0.25]

In [ ]:
border_north = 79
border_south = 73
border_east = 155
border_west = 120
borders = [border_north, border_south, border_east, border_west]

months = [7,8,9,10]

dates_wind, array_u10_filtered, array_v10_filtered, array_u10_upd_filtered, array_v10_upd_filtered = make_wind(borders, months, year, start_date, finish_date)

In [ ]:
u10_mean_neg, v10_mean, wind_mean, U_neg, V = mean_wind(array_u10_upd_filtered, array_v10_upd_filtered)
NB_delta_130E_km, U_Ekman, U_Ekman_sm, U_Ekman_sum, V_Ekman_sm, V_Ekman_sum, Ekman_data_u, Ekman_data_v = make_ekman(NB_delta_130E, U_neg, V)
U_neg_averages, V_averages, df = wind_3d(U_neg, V, NB_delta_130E_km)
print(df['a'].corr(df['b']))

In [ ]:
draw_border_wind(dates, NB_130E, NB_delta_130E_km, dates_wind, U_Ekman_sm, Ekman_data_u, str(year))

In [ ]:
# 2020
year = 2020
start_date = datetime.datetime(year, 7, 20).date()
finish_date = datetime.datetime(year, 10, 9).date()
dates = [(start_date + datetime.timedelta(days=i*3)) for i in range((finish_date - start_date).days // 3 + 1)]
NB_130E = [76.875,77.5,77.625,77.25,77.125,77.25,77.125,77.25,77.375,77.375,77.625,77.625,77.875,77.125,
           77.25,77.25,77.25,77.125,77.25,77.25,77.25,77.125,77,76.75,76.5,76.125,76.125,76.125]
NB_delta_130E = [np.nan,0.625,0.125,-0.375,-0.125,0.125,-0.125,0.125,0.125,0,0.25,0,0.25,-0.75,0.125,0,0,-0.125,0.125,0,0,-0.125,-0.125,-0.25,-0.25,-0.375,0,0]

In [ ]:
border_north = 79
border_south = 73
border_east = 155
border_west = 120
borders = [border_north, border_south, border_east, border_west]

months = [7,8,9,10]

dates_wind, array_u10_filtered, array_v10_filtered, array_u10_upd_filtered, array_v10_upd_filtered = make_wind(borders, months, year, start_date, finish_date)

In [ ]:
u10_mean_neg, v10_mean, wind_mean, U_neg, V = mean_wind(array_u10_upd_filtered, array_v10_upd_filtered)
NB_delta_130E_km, U_Ekman, U_Ekman_sm, U_Ekman_sum, V_Ekman_sm, V_Ekman_sum, Ekman_data_u, Ekman_data_v = make_ekman(NB_delta_130E, U_neg, V)
U_neg_averages, V_averages, df = wind_3d(U_neg, V, NB_delta_130E_km)
print(df['a'].corr(df['b']))

In [ ]:
draw_border_wind(dates, NB_130E, NB_delta_130E_km, dates_wind, U_Ekman_sm, Ekman_data_u, str(year))

In [ ]:
# 2021
year = 2021
start_date = datetime.datetime(year, 7, 14).date()
finish_date = datetime.datetime(year, 10, 12).date()
dates = [(start_date + datetime.timedelta(days=i*3)) for i in range((finish_date - start_date).days // 3 + 1)]
NB_130E = [74.25,74,74.25,74.5,74.875,75,75,75.25,75.375,75.625,75.25,75,75,75.375,75.5,75.875,
           76,76,76.125,76.25,76,76.25,76.375,76.625,76.5,76.375,76.25,76.375,76.375,76.375,76.125]
NB_delta_130E = [np.nan,-0.25,0.25,0.25,0.375,0.125,0,0.25,0.125,0.25,-0.375,-0.25,0,0.375,0.125,
                 0.375,0.125,0,0.125,0.125,-0.25,0.25,0.125,0.25,-0.125,-0.125,-0.125,0.125,0,0,-0.25]

In [ ]:
border_north = 78
border_south = 73
border_east = 155
border_west = 120
borders = [border_north, border_south, border_east, border_west]

months = [7,8,9,10]

dates_wind, array_u10_filtered, array_v10_filtered, array_u10_upd_filtered, array_v10_upd_filtered = make_wind(borders, months, year, start_date, finish_date)

In [ ]:
u10_mean_neg, v10_mean, wind_mean, U_neg, V = mean_wind(array_u10_upd_filtered, array_v10_upd_filtered)
NB_delta_130E_km, U_Ekman, U_Ekman_sm, U_Ekman_sum, V_Ekman_sm, V_Ekman_sum, Ekman_data_u, Ekman_data_v = make_ekman(NB_delta_130E, U_neg, V)
U_neg_averages, V_averages, df = wind_3d(U_neg, V, NB_delta_130E_km)
print(df['a'].corr(df['b']))

In [ ]:
draw_border_wind(dates, NB_130E, NB_delta_130E_km, dates_wind, U_Ekman_sm, Ekman_data_u, str(year))

In [ ]:
# 2022
year = 2022
start_date = datetime.datetime(year, 9, 21).date()
finish_date = datetime.datetime(year, 10, 18).date()
dates = [(start_date + datetime.timedelta(days=i*3)) for i in range((finish_date - start_date).days // 3 + 1)]
NB_130E = [77.625,77.625,77.875,77.375,77,75.625,75.5,76,75.875,75.625]
NB_delta_130E = [np.nan,0,0.25,-0.5,-0.375,-1.375,-0.125,0.5,-0.125,-0.25]

In [ ]:
border_north = 79
border_south = 75
border_east = 135
border_west = 125
borders = [border_north, border_south, border_east, border_west]

months = [9,10]

dates_wind, array_u10_filtered, array_v10_filtered, array_u10_upd_filtered, array_v10_upd_filtered = make_wind(borders, months, year, start_date, finish_date)

In [ ]:
u10_mean_neg, v10_mean, wind_mean, U_neg, V = mean_wind(array_u10_upd_filtered, array_v10_upd_filtered)
NB_delta_130E_km, U_Ekman, U_Ekman_sm, U_Ekman_sum, V_Ekman_sm, V_Ekman_sum, Ekman_data_u, Ekman_data_v = make_ekman(NB_delta_130E, U_neg, V)
U_neg_averages, V_averages, df = wind_3d(U_neg, V, NB_delta_130E_km)
print(df['a'].corr(df['b']))

In [ ]:
draw_border_wind(dates, NB_130E, NB_delta_130E_km, dates_wind, U_Ekman_sm, Ekman_data_u, str(year))

In [ ]:
# 2023
year = 2023
start_date = datetime.datetime(year, 8, 7).date()
finish_date = datetime.datetime(year, 10, 12).date()
dates = [(start_date + datetime.timedelta(days=i*3)) for i in range((finish_date - start_date).days // 3 + 1)]
NB_130E = [75.625,75.625,75.75,76,76.25,76.5,76.5,76.5,76.125,76.125,76.125,76.25,75.625,76.25,76.25,76,75,74.625,74.75,75.125,75.125,75.125,75.25]
NB_delta_130E = [np.nan,0,0.125,0.25,0.25,0.25,0,0,-0.375,0,0,0.125,-0.625,0.625,0,-0.25,-1,-0.375,0.125,0.375,0,0,0.125]

In [ ]:
border_north = 79
border_south = 73
border_east = 155
border_west = 120
borders = [border_north, border_south, border_east, border_west]

months = [8,9,10]

dates_wind, array_u10_filtered, array_v10_filtered, array_u10_upd_filtered, array_v10_upd_filtered = make_wind(borders, months, year, start_date, finish_date)

In [ ]:
u10_mean_neg, v10_mean, wind_mean, U_neg, V = mean_wind(array_u10_upd_filtered, array_v10_upd_filtered)
NB_delta_130E_km, U_Ekman, U_Ekman_sm, U_Ekman_sum, V_Ekman_sm, V_Ekman_sum, Ekman_data_u, Ekman_data_v = make_ekman(NB_delta_130E, U_neg, V)
U_neg_averages, V_averages, df = wind_3d(U_neg, V, NB_delta_130E_km)
print(df['a'].corr(df['b']))

In [ ]:
draw_border_wind(dates, NB_130E, NB_delta_130E_km, dates_wind, U_Ekman_sm, Ekman_data_u, str(year))